In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from sklearn.preprocessing import MinMaxScaler

In [3]:
# تنظیم مسیر ورودی (خروجی فاز ۲) و خروجی نهایی
INPUT_FILE = '../data/analyzed_data.csv'
OUTPUT_FILE = '../data/final_risk_data.csv'

# بارگذاری داده‌ها
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"✅ Data loaded. Shape: {df.shape}")
except FileNotFoundError:
    print("❌ File not found. Please run Phase 2 (Spatial Analysis) first.")

❌ File not found. Please run Phase 2 (Spatial Analysis) first.


In [4]:
print("Calculating sub-scores...")

# 1. امتیاز شدت تصادف (Severity)
# Fatal=3, Serious=2, Slight=1
severity_map = {'Fatal': 3, 'Serious': 2, 'Slight': 1}
# اگر مقدار نامشخص بود، کمترین شدت (1) را در نظر می‌گیریم
df['Severity_Score'] = df['Accident_Severity'].map(severity_map).fillna(1)

# 2. امتیاز شرایط محیطی (Environment)
# اگر هوا بارانی، برفی یا مه‌آلود باشد، امتیاز خطر بالا می‌رود
bad_weather_keywords = ['Raining', 'Snowing', 'Fog']
df['Weather_Score'] = df['Weather_Conditions'].apply(
    lambda x: 2 if any(k in str(x) for k in bad_weather_keywords) else 1
)

# اگر شب باشد و چراغی نباشد، خطر بیشتر است
df['Light_Score'] = df['Light_Conditions'].apply(
    lambda x: 2 if 'Darkness - no lighting' in str(x) else 1
)

# ترکیب امتیازات محیطی
df['Env_Score'] = df['Weather_Score'] + df['Light_Score']

print("✅ Severity and Environmental scores calculated.")

Calculating sub-scores...


NameError: name 'df' is not defined

In [ ]:
print("Calculating Final Risk Index...")

scaler = MinMaxScaler()

# نرمال‌سازی داده‌ها بین 0 تا 1 تا قابل مقایسه باشند
df['Norm_Severity'] = scaler.fit_transform(df[['Severity_Score']])
df['Norm_Density'] = scaler.fit_transform(df[['Density']]) # تراکم که از فاز قبل داریم
df['Norm_Env'] = scaler.fit_transform(df[['Env_Score']])

# امتیاز تشویقی برای نقاطی که داخل خوشه هستند (Cluster != -1)
df['Cluster_Bonus'] = df['Cluster'].apply(lambda x: 1 if x != -1 else 0)

# --- فرمول اصلی شاخص خطر ---
df['Risk_Index'] = (
    0.4 * df['Norm_Severity'] +
    0.3 * df['Norm_Density'] +
    0.2 * df['Norm_Env'] +
    0.1 * df['Cluster_Bonus']
)

# تبدیل شاخص به مقیاس ۰ تا ۱۰۰ برای خوانایی بهتر
df['Risk_Index'] = df['Risk_Index'] * 100

print("✅ Risk Index calculated successfully.")
print(df[['Accident_Severity', 'Density', 'Risk_Index']].head())

Calculating Final Risk Index...
✅ Risk Index calculated successfully.
  Accident_Severity   Density  Risk_Index
0             Fatal  0.388904   52.127648
1             Fatal  0.388904   52.127648
2             Fatal  1.367807   49.994863
3             Fatal  1.125135   58.044567
4             Fatal  1.621488   52.033640


In [ ]:
# نمایش ۱۰ تصادف با بیشترین شاخص خطر
top_risky = df.nlargest(10, 'Risk_Index')

print("Top 10 High-Risk Accidents:")
display(top_risky[['Date', 'Time', 'Accident_Severity', 'Weather_Conditions', 'Risk_Index']])

Top 10 High-Risk Accidents:


,Date,Time,Accident_Severity,Weather_Conditions,Risk_Index
44,2010-10-19,19:58:00,Fatal,Raining no high winds,77.800403
61,2010-12-23,11:05:00,Fatal,Fine no high winds,74.987844
14,2010-10-07,14:54:00,Fatal,Fine no high winds,73.008793
63,2010-07-14,13:35:00,Fatal,Raining no high winds,71.806697
72,2010-08-12,18:45:00,Fatal,Fine no high winds,71.618169
46,2010-01-01,04:26:00,Fatal,Fine no high winds,71.527682
50,2010-01-17,01:00:00,Fatal,Raining no high winds,70.387300
76,2010-10-07,17:29:00,Fatal,Fine no high winds,69.863095
104,2010-10-07,17:29:00,Fatal,Fine no high winds,69.863095
21,2010-09-03,01:30:00,Fatal,Fine no high winds,69.817315


In [ ]:
import os

# 1. رسم نقشه با استفاده از folium
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)

# فقط نقاط با ریسک بالا (برای سبکی نقشه)
high_risk_map_data = df[df['Risk_Index'] > 50]

for _, row in high_risk_map_data.iterrows():
    if row['Risk_Index'] > 80:
        color = 'black'
    elif row['Risk_Index'] > 65:
        color = 'red'
    else:
        color = 'orange'

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Risk: {row['Risk_Index']:.1f}"
    ).add_to(m)

# 2. ذخیره فایل HTML در همان پوشه (notebooks)
output_map_file = 'risk_map.html'
m.save(output_map_file)

print(f"✅ نقشه با موفقیت در کنار نوت‌بوک ذخیره شد: {os.path.abspath(output_map_file)}")


m

✅ نقشه با موفقیت در کنار نوت‌بوک ذخیره شد: c:\Users\Lenovo\Downloads\risk_map.html


In [ ]:
output_filename = 'risk_map.html'
m.save(output_filename)

print(f"✅ فایل ساخته شد در: {os.path.abspath(output_filename)}")

NameError: name 'm' is not defined

In [ ]:
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Final Risk Data saved to: {OUTPUT_FILE}")

✅ Final Risk Data saved to: ../data/final_risk_data.csv
